In [1]:
import mlflow
import pandas as pd
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType
from datetime import datetime, timedelta
from pydash import py_

from green_city.mlflow_config import get_mlflow_config
mlflow.set_tracking_uri(get_mlflow_config()["TRACKING_URI"])
client = MlflowClient()
experiment_id = client.get_experiment_by_name("green_city_experiments").experiment_id

run_ids = [r.run_id for r in client.list_run_infos(experiment_id, run_view_type=ViewType.ACTIVE_ONLY)]

all_runs = {run_id: client.get_run(run_id).to_dictionary() for run_id in run_ids}

In [2]:
runs_dict = {}

for id, details in all_runs.items():
    start_time = py_.get(details, "info.start_time")
    end_time = py_.get(details, "info.end_time")
    delta = round(end_time/1000 - start_time/1000, 2)
    runs_dict[id] = {
        "time_s": delta,
        **py_.get(details, "data.metrics"),
        **py_.get(details, "data.params")
    }

In [4]:
df = pd.DataFrame.from_dict(runs_dict, orient='index').astype({"datetime": "datetime64[ns]"})
df.index = df.index.rename("run_id")

df[df.feature=="net_load_kWh"]

,time_s,mae,mse,feature,pred_steps,building_nr,model,resolution,r2_score,datetime,index,train_steps,seasonalities
run_id,,,,,,,,,,,,,
1e31d575bc9d47779c5b6665e77dc9a9,12.01,8.442218,98.055765,net_load_kWh,24,5,TBATS,H,-3.113102,2009-01-03 22:00:00,8830,40,NaN
ef7b50f7d6d44a568edf6a4eab2f076c,9.66,3.324316,21.636353,net_load_kWh,24,5,TBATS,H,0.309809,2008-01-04 16:00:00,64,200,[24]
